<a href="https://colab.research.google.com/github/BSteiner1/Adversarial-Examples/blob/main/Notebooks/Differential_Evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random

In [2]:
# Load Fashion MNIST dataset and apply transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 26421880/26421880 [00:01<00:00, 16681402.62it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 304066.46it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5527064.49it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13328566.04it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:
labels_dict = {0 : "T-shirt/top",
               1 : "Trouser",
               2 : "Pullover",
               3 : "Dress",
               4 : "Coat",
               5 : "Sandal",
               6 : "Shirt",
               7 : "Sneaker",
               8 : "Bag",
               9 : "Ankle boot"}

In [79]:
def image_confidence(image, model, label):

    model.eval()

    with torch.no_grad():
        output = model(image.unsqueeze(0))

    # Get the confidence associated with the original prediction
    image_probabilities = F.softmax(output, dim=1)
    confidence = image_probabilities[0][label].item()

    #print(label)
    #print(image_probabilities)

    return confidence

In [63]:
def apply_mutation(image, mutation):

    x, y, p = mutation

    image[0][int(x)][int(y)] = p

    return image

In [69]:
m1 = np.array([0,0,0.])
m2 = np.array([6,0,1.])
img = test_dataset[0][0]

#print(img)

par_img = img.clone()
ch_img = img.clone()


par = apply_mutation(par_img, m1)
ch = apply_mutation(ch_img, m2)

#print(par)
#print(ch)
#par[0] == ch[0]

In [11]:
# Define the CNN model using nn.Sequential
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(32, 64, kernel_size=3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Flatten(),
    nn.Linear(3136, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
model.load_state_dict(torch.load('/content/drive/MyDrive/MA498/CNN_model.pth'))
model.eval()  # Set the model to evaluation mode

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=3136, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=10, bias=True)
)

In [123]:
def differential_evolution(tensor, model):

  model.eval()
  image, label = tensor

  population = [np.array([random.randint(0,27), random.randint(0,27), random.uniform(-1,1)]) for _ in range(400)]

  for iter in range(100):
      if iter % 5 == 0:
          print(iter)
      for j in range(400):

          #global swap
          #swap = False

          parent_mutation = population[j]
          candidates = [i for i in range(400) if i != j]
          a, b, c = np.random.choice(candidates, 3, replace=False)

          # Create mutant vector and clip if necessary
          child_mutation = population[a] + 0.5 * (population[b] - population[c])
          child_mutation[:2] = np.round(child_mutation[:2]).astype(int)
          child_mutation = np.clip(child_mutation, [0, 0, -1.], [27, 27, 1.])

          # print(parent_mutation)
          # print(child_mutation)

          # def apply_mutation(image, mutation):

          #     x, y, p = mutation
          #     image[0][int(x)][int(y)] = p

          #     #print(image)

          #     return image

          parent = image.clone()
          child = image.detach().clone()

          parent = apply_mutation(parent, parent_mutation)
          child = apply_mutation(child, child_mutation)

          # print(parent)
          # print(child)

          # print(parent[0] == child[0])

          def fitness(parent, child, label):
              parent_confidence =  image_confidence(parent, model, label)
              child_confidence =  image_confidence(child, model, label)
              #print("Child", child_confidence)
              #print("Parent", parent_confidence)

              if child_confidence < parent_confidence:
                swap = True
                #print(swap)
                print(child_confidence)
                return child, swap
              else:
                swap = False
                return parent, swap

          fittest, swap = fitness(parent, child, label)
          #print(swap)
          if swap == True:
            #print("swap")
            population[j] = child_mutation
          else:
            None

  return population

In [124]:
img = test_dataset[0]
solution = differential_evolution(img, model)

0
0.9971452355384827
0.9989107847213745
0.9989939332008362
0.9986489415168762
0.9980744123458862
0.996325671672821
0.9989703893661499
0.9983910322189331
0.9987912774085999
0.9987568855285645
0.998880922794342
0.9989388585090637
0.9987738728523254
0.9989305138587952
0.9987302422523499
0.999030590057373
0.9986431002616882
0.997296154499054
0.9989709854125977
0.9927052855491638
0.998984158039093
0.9989823698997498
0.9988119602203369
0.9988792538642883
0.9989907145500183
0.9990971088409424
0.9988086223602295
0.9988635778427124
0.9975679516792297
0.9989270567893982
0.9987691044807434
0.9991229176521301
0.9987236857414246
0.9977130889892578
0.9989835619926453
0.9986183643341064
0.9982151985168457
0.996769905090332
0.9989318251609802
0.9989509582519531
0.9985306262969971
0.9842445254325867
0.9990277290344238
0.9965357780456543
0.9989797472953796
0.99896240234375
0.998414158821106
0.9986937642097473
0.9989308714866638
0.9988235831260681
0.998866081237793
0.9912508726119995
0.9989688396453857
0

KeyboardInterrupt: 

In [175]:
population = [np.array([random.randint(0,28), random.randint(0,28), random.uniform(-1,1)]) for _ in range(400)]
a, b, c = 10, 40, 80
mutant_vector = population[a] + 0.5 * (population[b] - population[c])